In [1]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_API_BASE"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION")
)
openai_chatmodel = os.getenv("AZURE_OPENAI_CHAT_MODEL")

In [2]:
query = "What is the homepage of Vlad Iliescu? What is the content of that page?"

completion = client.chat.completions.create(
    model=openai_chatmodel,
    messages=[{"role": "user", "content": query}],
    temperature=0.0
)

print(completion.choices[0].message.content)



As of my last knowledge update in October 2023, I do not have specific information about Vlad Iliescu's homepage or its content. If you are looking for details about a specific individual, I recommend performing a web search using their name to find their official website or online profiles. Always ensure you are accessing legitimate and verified sources.


In [7]:

import html
import re
from html.parser import HTMLParser
import json
import requests
from bs4 import BeautifulSoup
import urllib

GRAY = "\033[90m"
BOLD = "\033[1m"
RESET = "\033[0m"


def get_search_results_for(query):
    encoded_query = urllib.parse.urlencode({'q': query})
    url = f'https://html.duckduckgo.com/html?q={encoded_query}'

    request = urllib.request.Request(url)
    request.add_header('User-Agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')

    raw_response = urllib.request.urlopen(request).read()
    html = raw_response.decode("utf-8")

    soup = BeautifulSoup(html, 'html.parser')
    a_results = soup.select("a.result__a")

    links = []
    for a_result in a_results:
        # print(a_result)
        url = a_result.attrs['href']
        title = a_result.text
        links.append({"title": title,  "url": url} )

    return links

class HTMLToTextParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.text = []
        self.ignore_tags = {'script', 'style', 'meta', 'link'}
        self.current_tag = None

    def handle_starttag(self, tag, attrs):
        self.current_tag = tag
        if tag in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
            self.text.append(f"\n{'#' * int(tag[1])} ")
        elif tag == 'p':
            self.text.append('\n\n')
        elif tag == 'br':
            self.text.append('\n')
        elif tag == 'li':
            self.text.append('\n- ')

    def handle_endtag(self, tag):
        if tag in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
            self.text.append('\n')
        self.current_tag = None

    def handle_data(self, data):
        if self.current_tag not in self.ignore_tags:
            self.text.append(data.strip())

    def get_text(self):
        result = ''.join(self.text)
        # Clean up extra whitespace
        result = re.sub(r'\n\s*\n', '\n\n', result)
        return html.unescape(result.strip())

def html_to_simple_text(html_content):
    parser = HTMLToTextParser()
    parser.feed(str(html_content))
    return parser.get_text()


def load_page_content(url) -> str:
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'})
    page_content = response.content.decode('utf-8')

    soup = BeautifulSoup(page_content, 'html.parser')
    for element in soup(['script', 'style', 'meta', 'link']):
        element.decompose()

    return html_to_simple_text(soup)

def get_homepage(blogger: str) -> str:
    return "https://vladiliescu.net"
    # return get_search_results_for(f"{blogger} homepage")[0]["url"]


tools = [{
        "type": "function",
        "function": {
            "name": "get_homepage",
            "description": "Returns the homepage of a particular blogger.",
            "parameters": {
                "type": "object",
                "properties": {
                    "blogger": {
                        "type": "string",
                        "description": "Blogger name"
                    }
                },
                "required": [
                    "blogger"
                ],
                "additionalProperties": False
            },
            "strict": True
        }
    },
    {
        "type": "function",
        "function": {
            "name": "load_page_content",
            "description": "Returns the content of a particular webpage.",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "Url of the webpage for which to retrieve the content"
                    }
                },
                "required": [
                    "url"
                ],
                "additionalProperties": False
            },
            "strict": True
        }
    }
]

def call_function(name, args):
    if name == "get_homepage":
        return get_homepage(**args)
    if name == "load_page_content":
        return load_page_content(**args)



messages = [{"role": "user", "content": query}]

total_input_token_count = 0
total_output_token_count = 0

while (True):
    completion = client.chat.completions.create(
        model=openai_chatmodel,
        messages=messages,
        tools=tools
    )

    total_input_token_count += completion.usage.prompt_tokens
    total_output_token_count += completion.usage.completion_tokens

    if completion.choices[0].finish_reason == "stop":
        print(f"{BOLD}Final answer: {completion.choices[0].message.content}{RESET}")
        break
    elif completion.choices[0].finish_reason == "tool_calls":
        messages.append(completion.choices[0].message)
        for tool_call in completion.choices[0].message.tool_calls:
            name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)

            result = call_function(name, args)
            print(f"Called {BOLD}{name}({args}){RESET} and it returned {GRAY}{result[:300]}{RESET}")

            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result
            })
    else:
        raise Exception("We're not supposed to be here")


Called get_homepage({'blogger': 'Vlad Iliescu'}) and it returned https://vladiliescu.net
Called load_page_content({'url': 'https://vladiliescu.net'}) and it returned Vlad IliescuVlad Iliescu
- ✨Awesome✨ Gen AI
- Search
- Talks 🎙
- Apps
- Wiki 📚
- Archive
# Hi, I’m Vlad 👋

I’m a software & AI architect,founder, andMicrosoft MVP on AI.
Iwriteandspeakabout machine learning in general and AI in particular. Follow me onSubstackor plain oldRSS.

Here are some of my hi
Final answer: The homepage of Vlad Iliescu is [https://vladiliescu.net](https://vladiliescu.net).

The content of the page includes an introduction to Vlad Iliescu, who is a software and AI architect, founder, and AI enthusiast, holding the Microsoft MVP title. His site details his contributions to AI research, applications, and various projects. It also includes blog posts, guides, and updates on various topics such as:

- His work in co-founding AI ventures like **ZenAIos** (in the medical sector) and **NRGI.ai** (focused on 